In [ ]:
# importation de quelques librairies

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# chemin de nos données d'entrainement et de test

train_path = '../data/data_reg/wine_train.csv' # replace with your path
test_path = '../data/data_reg/wine_test.csv' # replace with your path

# chargement des données

df_train = pd.read_csv(train_path).drop('wine_ID', axis=1)
df_test = pd.read_csv(test_path)

## Analyse des données

In [ ]:
# visualisation des données

df_train.head()

In [ ]:
# informations sur les données

df_train.info()

In [ ]:
# description des données

df_train.describe()

In [ ]:
# forme des données

print(df_train.shape, df_test.shape)

In [ ]:
# Disrtibution des classes

df_train['target'].hist()
plt.title('Quality distribution')
plt.xlabel('Quality')

In [ ]:
# Matrice de corrélation

plt.figure(figsize=(10, 10))
sns.heatmap(df_train.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')

In [ ]:
# Distribution des variables

fig = plt.figure(figsize=(20, 20))
for i in range(len(df_train.columns)):
    fig.add_subplot(5, 3, i+1)
    sns.histplot(df_train.iloc[:, i], color='green', label=df_train.columns[i])
    # show the mean and median
    plt.axvline(df_train.iloc[:, i].mean(), linestyle='dashed', color='red', label='mean')
    plt.axvline(df_train.iloc[:, i].median(), linestyle='dashed', color='blue', label='median')
    plt.legend()

## Séparation train-test

In [ ]:
# Séparation des données en train test

X = df_train.drop('target', axis=1)
y = df_train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

# Entrainement du modèle

model.fit(X_train, y_train)

# Prédiction sur les données de test

y_preds = model.predict(X_test)

# Evaluation du modèle

print(f"Le r2 vaut : {r2_score(y_test, y_preds)}")

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds})

new_results.to_csv('../résultats_models/regression/linear_regression.csv', index=False)

# soumission des résultats

model = LinearRegression()

model.fit(X, y)

y_preds = model.predict(df_test.drop('wine_ID', axis=1))

submission = pd.DataFrame({'wine_ID': df_test['wine_ID'],
                            'target': y_preds})

submission.to_csv('../soumission/regression/linear_regression.csv', index=False)



## CatBoost Regressor

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=10000, depth=10, learning_rate=0.1, loss_function='RMSE', eval_metric='R2', random_seed=42)

# Entrainement du modèle

model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=0)

# Prédiction sur les données de test

y_preds = model.predict(X_test)

# Evaluation du modèle

print(f'Le r2 vaut : {r2_score(y_test, y_preds)}')

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds})

new_results.to_csv('../résultats_models/regression/catboost.csv', index=False)

# soumission des résultats

model = CatBoostRegressor(iterations=10000, depth=10, learning_rate=0.1, loss_function='RMSE', eval_metric='R2', random_seed=42)

model.fit(X, y, verbose=0)

y_preds = model.predict(df_test.drop('wine_ID', axis=1))

submission = pd.DataFrame({'wine_ID': df_test['wine_ID'],
                            'target': y_preds})

submission.to_csv('../soumission/regression/catboost.csv', index=False)

## Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Définition de notre espaces de paramètres

params = {'n_estimators': [100, 200, 300, 400, 500],
            'max_depth': [5, 10, 15, 20, 25],
            'min_samples_split': [2, 5, 10, 15, 20],
            'min_samples_leaf': [1, 2, 5, 10, 15]}

# Création de notre modèle

model = RandomForestRegressor(random_state=42)

# Création de notre grille de recherche

grid = GridSearchCV(model, params, cv=5, scoring='r2', n_jobs=-1)

# Entrainement de notre modèle

grid.fit(X_train, y_train)

# Affichage des meilleurs paramètres

print(grid.best_params_)
print(grid.best_score_)

# Prédiction sur les données de test

y_preds = grid.predict(X_test)

# Evaluation du modèle

print(f'Le r2 vaut : {r2_score(y_test, y_preds)}')

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds})

new_results.to_csv('../résultats_models/regression/random_forest.csv', index=False)

# soumission des résultats

model = RandomForestRegressor(random_state=42)
model.set_params(**grid.best_params_)

model.fit(X, y)

y_preds = model.predict(df_test.drop('wine_ID', axis=1))

submission = pd.DataFrame({'wine_ID': df_test['wine_ID'],
                            'target': y_preds})

submission.to_csv('../soumission/regression/random_forest.csv', index=False)

## Normalisation des données

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

df_train_scaled = scaler.fit_transform(df_train.drop('target', axis=1))
df_test_scaled = scaler.transform(df_test.drop('wine_ID', axis=1))

## Linear Regression

In [ ]:
model = LinearRegression()

# Entrainement du modèle

model.fit(X_train_scaled, y_train)

# Prédiction sur les données de test

y_preds = model.predict(X_test_scaled)

# Evaluation du modèle

print(f"Le r2 vaut : {r2_score(y_test, y_preds)}")

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds})

new_results.to_csv('../résultats_models/regression/linear_regression_scaled.csv', index=False)

# soumission des résultats

model = LinearRegression()

model.fit(df_train_scaled, y)

y_preds = model.predict(df_test_scaled)

submission = pd.DataFrame({'wine_ID': df_test['wine_ID'],
                            'target': y_preds})

submission.to_csv('../soumission/regression/linear_regression_scaled.csv', index=False)

## CatBoost Regressor

In [ ]:
model = CatBoostRegressor(iterations=10000, depth=10, learning_rate=0.1, loss_function='RMSE', eval_metric='R2', random_seed=42)

# Entrainement du modèle

model.fit(X_train_scaled, y_train, eval_set=(X_test_scaled, y_test), verbose=0)

# Prédiction sur les données de test

y_preds = model.predict(X_test_scaled)

# Evaluation du modèle

print(f'Le r2 vaut : {r2_score(y_test, y_preds)}')

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds})

new_results.to_csv('../résultats_models/regression/catboost_scaled.csv', index=False)

# soumission des résultats

model = CatBoostRegressor(iterations=10000, depth=10, learning_rate=0.1, loss_function='RMSE', eval_metric='R2', random_seed=42)

model.fit(df_train_scaled, y, verbose=0)

y_preds = model.predict(df_test_scaled)

submission = pd.DataFrame({'wine_ID': df_test['wine_ID'],
                            'target': y_preds})

submission.to_csv('../soumission/regression/catboost_scaled.csv', index=False)

## Random Forest Regressor

In [ ]:
model = RandomForestRegressor(random_state=42)

# Définition de notre espaces de paramètres

params = {'n_estimators': [100, 200, 300, 400, 500],
            'max_depth': [5, 10, 15, 20, 25],
            'min_samples_split': [2, 5, 10, 15, 20],
            'min_samples_leaf': [1, 2, 5, 10, 15]}

# Création de notre grille de recherche

grid = GridSearchCV(model, params, cv=5, scoring='r2', n_jobs=-1)

# Entrainement de notre modèle

grid.fit(X_train_scaled, y_train)

# Affichage des meilleurs paramètres

print(grid.best_params_)
print(grid.best_score_)

# Prédiction sur les données de test

y_preds = grid.predict(X_test_scaled)

# Evaluation du modèle

print(f'Le r2 vaut : {r2_score(y_test, y_preds)}')

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds})

new_results.to_csv('../résultats_models/regression/random_forest_scaled.csv', index=False)

# soumission des résultats

model = RandomForestRegressor(random_state=42)
model.set_params(**grid.best_params_)

model.fit(df_train_scaled, y)

y_preds = model.predict(df_test_scaled)

submission = pd.DataFrame({'wine_ID': df_test['wine_ID'],
                            'target': y_preds})

submission.to_csv('../soumission/regression/random_forest_scaled.csv', index=False)